# 📈 开源协作网络时序分析
## 核心目标：从时序数据中提取关键洞察，为DataEase可视化准备数据

本Notebook将演示：
1. 加载并探索时序协作数据
2. 分析网络结构月度演化
3. 识别关键贡献者与社区变化
4. 输出可视化友好格式的数据

## 第一步：环境准备与数据加载

In [ ]:
import sys
import os
sys.path.append('../src')  # 添加src目录到路径，方便导入自定义模块

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 设置中文字体和美观样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
sns.set_palette("husl")

print("✅ 环境准备完成")

In [ ]:
# 加载已经生成的数据
print("加载数据文件...")

developers_df = pd.read_csv('../data/developers.csv')
collab_df = pd.read_csv('../data/collaborations_temporal.csv')
monthly_df = pd.read_csv('../data/monthly_metrics.csv')
latest_network_df = pd.read_csv('../data/latest_network.csv')

print(f"✅ 开发者数据: {len(developers_df)} 位开发者")
print(f"✅ 协作记录: {len(collab_df)} 条时序记录")
print(f"✅ 月度指标: {len(monthly_df)} 个月份")
print(f"✅ 最新网络: {len(latest_network_df)} 条边")

# 显示数据预览
print("\n开发者数据预览:")
display(developers_df.head())

print("\n协作数据预览 (含时间戳):")
collab_df['timestamp'] = pd.to_datetime(collab_df['timestamp'])
display(collab_df.head())

## 第二步：宏观趋势分析（制作第一张图表）

In [ ]:
# 创建第一个可视化：协作活动趋势
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('开源协作网络宏观趋势分析', fontsize=16, fontweight='bold')

# 1. 月度协作数量趋势
ax1 = axes[0, 0]
monthly_df.plot(x='year_month', y='num_collaborations', kind='line', 
                marker='o', ax=ax1, color='#2E86AB', linewidth=2)
ax1.set_title('月度协作数量变化', fontsize=12)
ax1.set_xlabel('年月')
ax1.set_ylabel('协作次数')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# 2. 活跃开发者数量趋势
ax2 = axes[0, 1]
monthly_df.plot(x='year_month', y='num_active_developers', kind='area',
                alpha=0.6, ax=ax2, color='#A23B72')
ax2.set_title('活跃开发者数量变化', fontsize=12)
ax2.set_xlabel('年月')
ax2.set_ylabel('活跃开发者数')
ax2.tick_params(axis='x', rotation=45)

# 3. 技术栈分布（条形图）
ax3 = axes[1, 0]
tech_dist = developers_df['primary_tech'].value_counts()
tech_dist.plot(kind='bar', ax=ax3, color='#F18F01')
ax3.set_title('开发者技术栈分布', fontsize=12)
ax3.set_xlabel('技术栈')
ax3.set_ylabel('开发者数量')
ax3.tick_params(axis='x', rotation=45)

# 4. 协作权重分布（箱型图）
ax4 = axes[1, 1]
monthly_df['avg_collab_weight'].plot(kind='box', ax=ax4, color='#73AB84')
ax4.set_title('协作强度（权重）分布', fontsize=12)
ax4.set_ylabel('平均协作权重')

plt.tight_layout()
plt.savefig('../data/monthly_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 已生成宏观趋势分析图表，保存为 data/monthly_trends.png")

## 第三步：网络结构分析（使用我们的智能引擎）

In [ ]:
# 导入我们昨天写的智能图引擎
from graph_engine import GraphEngine, test_engine

# 测试引擎
engine = test_engine()
engine_info = engine.get_info()
print(f"当前使用引擎: {engine_info['backend']} (版本: {engine_info['version']})")

In [ ]:
# 构建最新月份的网络图
print("\n构建最新月份协作网络...")
G = engine.create_graph(directed=True)

# 添加节点（开发者）
for _, dev in developers_df.iterrows():
    G.add_node(dev['developer_id'], 
               name=dev['name'], 
               tech=dev['primary_tech'], 
               activity=dev['activity_level'])

# 添加边（协作关系）
for _, edge in latest_network_df.iterrows():
    G.add_edge(edge['source'], edge['target'], weight=edge['weight'])

print(f"网络构建完成: {G.number_of_nodes()} 个节点, {G.number_of_edges()} 条边")

# 计算网络指标
print("\n计算网络关键指标...")
pagerank = engine.calculate_pagerank(G)
degree_centrality = engine.calculate_degree_centrality(G)

# 找出最重要的5个开发者
top_developers = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:5]
print("🏆 关键贡献者排名 (PageRank):")
for dev_id, score in top_developers:
    dev_info = developers_df[developers_df['developer_id'] == dev_id].iloc[0]
    print(f"   第{top_developers.index((dev_id, score))+1}名: {dev_info['name']} "
          f"({dev_info['primary_tech']}), 得分: {score:.4f}")

## 第四步：社区演化分析（核心洞察）

In [ ]:
# 分析社区结构（每月）
print("分析月度社区结构演化...")

# 为每个月份构建网络并检测社区
community_evolution = []

# 按月份分组
monthly_groups = collab_df.groupby('year_month')

for month, month_data in monthly_groups:
    # 创建该月的网络
    G_month = engine.create_graph(directed=True)
    
    # 添加该月活跃的开发者
    active_devs = set(month_data['source']).union(set(month_data['target']))
    for dev_id in active_devs:
        dev_info = developers_df[developers_df['developer_id'] == dev_id]
        if not dev_info.empty:
            G_month.add_node(dev_id, 
                           name=dev_info.iloc[0]['name'],
                           tech=dev_info.iloc[0]['primary_tech'])
    
    # 添加该月的协作关系
    for _, edge in month_data.iterrows():
        G_month.add_edge(edge['source'], edge['target'], weight=edge['weight'])
    
    # 检测社区（转换为无向图）
    if G_month.number_of_edges() > 0:
        communities = engine.add_community_detection(G_month)
        
        # 记录社区信息
        for comm_id, comm_nodes in enumerate(communities):
            for node in comm_nodes:
                community_evolution.append({
                    'year_month': month,
                    'developer_id': node,
                    'community_id': comm_id,
                    'community_size': len(comm_nodes)
                })
    
    print(f"  {month}: {len(active_devs)}位活跃开发者, {len(communities) if 'communities' in locals() else 0}个社区")

# 转换为DataFrame
community_df = pd.DataFrame(community_evolution)
print(f"\n✅ 社区演化数据: {len(community_df)} 条记录")

In [ ]:
# 可视化社区演化
# 准备桑基图数据（展示开发者社区间流动）

# 1. 社区数量变化
comm_counts = community_df.groupby('year_month')['community_id'].nunique().reset_index()
comm_counts.columns = ['year_month', 'num_communities']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 社区数量趋势
ax1.plot(comm_counts['year_month'], comm_counts['num_communities'], 
         marker='s', color='#6A0572', linewidth=2)
ax1.fill_between(comm_counts['year_month'], comm_counts['num_communities'], 
                 alpha=0.2, color='#6A0572')
ax1.set_title('社区数量变化趋势', fontsize=12)
ax1.set_xlabel('年月')
ax1.set_ylabel('社区数量')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# 社区规模分布（箱型图）
comm_sizes = community_df.groupby(['year_month', 'community_id'])['community_size'].first()
monthly_size_stats = comm_sizes.groupby('year_month').agg(['mean', 'std']).reset_index()

ax2.errorbar(range(len(monthly_size_stats)), 
             monthly_size_stats['mean'], 
             yerr=monthly_size_stats['std'],
             fmt='o-', color='#119DA4', 
             ecolor='#FF715B', capsize=5)
ax2.set_title('社区平均规模变化（带标准差）', fontsize=12)
ax2.set_xlabel('时间序列')
ax2.set_ylabel('平均社区规模')
ax2.set_xticks(range(len(monthly_size_stats)))
ax2.set_xticklabels(monthly_size_stats['year_month'], rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/community_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 已生成社区演化分析图表，保存为 data/community_evolution.png")

## 第五步：为DataEase准备优化数据

In [ ]:
print("为DataEase可视化准备优化格式的数据...")

# 1. 节点数据（开发者信息 + PageRank分数）
node_data = developers_df.copy()

# 添加PageRank分数
pagerank_df = pd.DataFrame(list(pagerank.items()), 
                           columns=['developer_id', 'pagerank_score'])
node_data = pd.merge(node_data, pagerank_df, on='developer_id', how='left')

# 添加度中心性
degree_df = pd.DataFrame(list(degree_centrality.items()), 
                         columns=['developer_id', 'degree_centrality'])
node_data = pd.merge(node_data, degree_df, on='developer_id', how='left')

# 保存节点数据
node_data.to_csv('../data/for_viz_nodes.csv', index=False)
print(f"✅ 节点数据: ../data/for_viz_nodes.csv ({len(node_data)}行)")

# 2. 边数据（协作关系）
edge_data = latest_network_df.copy()
# 添加技术栈信息
edge_data = pd.merge(edge_data, 
                     developers_df[['developer_id', 'primary_tech']], 
                     left_on='source', right_on='developer_id', 
                     how='left')
edge_data = edge_data.rename(columns={'primary_tech': 'source_tech'})
edge_data = pd.merge(edge_data, 
                     developers_df[['developer_id', 'primary_tech']], 
                     left_on='target', right_on='developer_id', 
                     how='left')
edge_data = edge_data.rename(columns={'primary_tech': 'target_tech'})
edge_data = edge_data.drop(columns=['developer_id_x', 'developer_id_y'])

# 标记技术栈是否匹配
edge_data['tech_match'] = edge_data['source_tech'] == edge_data['target_tech']

edge_data.to_csv('../data/for_viz_edges.csv', index=False)
print(f"✅ 边数据: ../data/for_viz_edges.csv ({len(edge_data)}行)")

# 3. 时序指标数据（已准备好）
monthly_df.to_csv('../data/for_viz_trends.csv', index=False)
print(f"✅ 趋势数据: ../data/for_viz_trends.csv ({len(monthly_df)}行)")

# 4. 社区演化数据
community_df.to_csv('../data/for_viz_communities.csv', index=False)
print(f"✅ 社区数据: ../data/for_viz_communities.csv ({len(community_df)}行)")

print("\n🎉 数据准备完成！现在可以导入DataEase进行可视化。")

## 总结

我们已经完成了：
1. ✅ 数据探索与宏观趋势分析
2. ✅ 网络结构分析（使用智能引擎）
3. ✅ 社区演化分析
4. ✅ 为DataEase准备了4个优化数据集

**下一步：使用DataEase创建交互式仪表盘**